<a href="https://colab.research.google.com/github/yunn333/AI/blob/main/HW0401.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.引入老師的基礎四套件

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. 安裝並執行 Ollama

先到官網找到安裝程式並安裝。

In [38]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


###3.啟用ollama伺服器
在伺服器上拉一個
gemma3:1b

In [39]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [40]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 4. 用 OpenAI API 使用

導入open AI、API金鑰及API網址

In [41]:
import openai
from openai import OpenAI

In [42]:
api_key = "ollama"

In [43]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 5. 使用Gradio創造聊天機器人網頁
設定自己的聊天機器人與頁面，並開始應用

In [44]:
!pip install gradio

In [45]:
import gradio as gr

In [46]:
title = "多益機器人"
description = """
你是一位英語訓練專家，專精於多益（TOEIC）考試訓練，擁有豐富的語言教學經驗與題庫知識。
你的任務是分析使用者的英文弱點，並根據多益考試的題型（Part 1～Part 7）量身打造練習題目，
每題附上標準答案與詳細講解，幫助使用者理解語法、單字用法與解題技巧。

請根據使用者的回答進行以下任務：
1. 判斷錯誤類型與弱點（如：文法、單字、句構、閱讀理解等）。
2. 出題：根據弱點與程度產生符合多益題型的練習題（可單題或多題）。
3. 講解：提供正確答案、錯誤選項說明，以及語法與詞彙解析。
4. 建議：給予個人化的學習建議與下一步練習方向。

語氣需清楚、有條理，具備鼓勵性與教學性質。適合中階英語學習者閱讀理解。
必要時可中英對照講解，或用全中文解釋複雜觀念。

請主動引導使用者開始練習，並在每次互動中持續協助其英文能力進步。
"""
description = "你好，我是你的專屬英語教練，只要你有空，990分不是夢。"
model = "gemma3:1b"

In [47]:
messages = [{"role":"system", "content":system}]
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [48]:
state = gr.State(messages)

In [49]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [50]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://43f65ed88f939f09d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
